In [ ]:
!pip install matplotlib google-cloud-storage

In [ ]:
import collections
import copy
import hashlib
import io
import os
import subprocess
import textwrap
import time
import glob

from typing import List, Text

from PIL import Image

import cv2
import numpy as np
import pandas as pd
import tabulate
import seaborn as sns
from tqdm import tqdm
import ast


import tensorflow as tf

import matplotlib.pyplot as plt

In [ ]:
from google.colab import auth

# Authenticate user for access. There will be a popup asking you to sign in with your user and approve access.
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Global params

class Globals:
  # GCP project with GCS bucket of interest
  gcp_project = 'dx-scin-public' #@param

  # GCS bucket with data to read
  gcs_bucket_name = 'dx-scin-public-data' #@param

  # CSV of case metadata to read
  cases_csv = 'dataset/scin_cases.csv' #@param

  # CSV of label metadata to read
  labels_csv = 'dataset/scin_labels.csv' #@param

  # Images directory
  gcs_images_dir = 'dataset/images/' #@param

  ### Key column names
  image_path_columns = ['image_1_path', 'image_2_path', 'image_3_path']
  weighted_skin_condition_label = "weighted_skin_condition_label"
  skin_condition_label = "dermatologist_skin_condition_on_label_name"

  ###### Formed during execution:

  # Client for querying GCS
  gcs_storage_client = None

  # Bucket object for loading files
  gcs_bucket = None

  # pd.DataFrame for the loaded metadata_csv
  cases_df = None

  # pd.DataFrame for the loaded labels_csv
  cases_and_labels_df = None

print(f'GCS bucket name: {Globals.gcs_bucket_name}')
print(f'cases_csv: {Globals.cases_csv}')
print(f'labels_csv: {Globals.labels_csv}')
print(f'images dir: {Globals.gcs_images_dir}')

GCS bucket name: dx-scin-public-data
cases_csv: dataset/scin_cases.csv
labels_csv: dataset/scin_labels.csv
images dir: dataset/images/


## Load dataset

In [ ]:
#@title Create a dataframe that contains the metadata and condition labels

from google.cloud import storage

def list_blobs(storage_client, bucket_name):
  """Helper to list blobs in a bucket (useful for debugging)."""
  blobs = storage_client.list_blobs(bucket_name)
  for blob in blobs:
    print(blob)

def initialize_df_with_metadata(bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  df['case_id'] = df['case_id'].astype(str)
  return df

def augment_metadata_with_labels(df, bucket, csv_path):
  """Loads the given CSV into a pd.DataFrame."""
  labels_df = pd.read_csv(io.BytesIO(bucket.blob(csv_path).download_as_string()), dtype={'case_id': str})
  labels_df['case_id'] = labels_df['case_id'].astype(str)
  merged_df = pd.merge(df, labels_df, on='case_id')
  return merged_df

Globals.gcs_storage_client = storage.Client(Globals.gcp_project)
Globals.gcs_bucket = Globals.gcs_storage_client.bucket(
    Globals.gcs_bucket_name
)
Globals.cases_df = initialize_df_with_metadata(Globals.gcs_bucket, Globals.cases_csv)
Globals.cases_and_labels_df = augment_metadata_with_labels(Globals.cases_df, Globals.gcs_bucket, Globals.labels_csv)
print(len(Globals.cases_and_labels_df))

5033


In [ ]:
Globals.cases_and_labels_df.columns

Index(['case_id', 'source', 'release', 'year', 'age_group', 'sex_at_birth',
       'fitzpatrick_skin_type',
       'race_ethnicity_american_indian_or_alaska_native',
       'race_ethnicity_asian', 'race_ethnicity_black_or_african_american',
       'race_ethnicity_hispanic_latino_or_spanish_origin',
       'race_ethnicity_middle_eastern_or_north_african',
       'race_ethnicity_native_hawaiian_or_pacific_islander',
       'race_ethnicity_white', 'race_ethnicity_other_race',
       'race_ethnicity_prefer_not_to_answer', 'textures_raised_or_bumpy',
       'textures_flat', 'textures_rough_or_flaky', 'textures_fluid_filled',
       'body_parts_head_or_neck', 'body_parts_arm', 'body_parts_palm',
       'body_parts_back_of_hand', 'body_parts_torso_front',
       'body_parts_torso_back', 'body_parts_genitalia_or_groin',
       'body_parts_buttocks', 'body_parts_leg', 'body_parts_foot_top_or_side',
       'body_parts_foot_sole', 'body_parts_other',
       'condition_symptoms_bothersome_appearan

In [ ]:
Globals.cases_and_labels_df.sample(1)

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us
1451,-5724889583050499613,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NaN,NaN,NaN,NaN,...,NO,NaN,NaN,NaN,NaN,NaN,True,True,2.0,2.0


## Important Functions

**Variables**
* df_original: The full, unmodified dataset containing all cases, metadata, and image paths. This is equivalent to Globals.cases_and_labels_df and can be used like a normal pandas DataFrame.
* df_filtered: A working copy of the dataset that you can safely modify, filter, or clean without affecting the original.
* image_dir : Use this directory to access images within google cloud


**Functions**
* read_image_from_gcs(gcs_path)
  - Downloads and decodes an image directly from your GCS bucket using the path stored in the dataset (e.g. "dataset/images/12345.png").
*  get_all_image_paths(df):
  - Extracts all unique image paths from the three image columns (image_1_path, image_2_path, image_3_path) in the dataset.
* show_case_images(case_id)
  - Displays all available images for a given case_id directly from GCS.
* convert_to_binary_var(col_name)
  - convert to binary values

  Note: add instructions if we want to analyze/change/decode all images together

In [ ]:
df_original = Globals.cases_and_labels_df

In [ ]:
df_filtered = df_original.copy()

In [ ]:
image_dir = "/content/gcs_mount/dataset/images"

In [ ]:
def read_image_from_gcs(gcs_path):
    """Reads and decodes an image from GCS (relative path)."""
    try:
        blob = Globals.gcs_bucket.blob(gcs_path)
        img_bytes = blob.download_as_bytes()
        img_array = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
        if img is None:
            return None
        # Convert BGR (OpenCV) → RGB (matplotlib)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        return img_rgb
    except Exception as e:
        print(f"Failed to read {gcs_path}: {e}")
        return None


In [ ]:
def get_all_image_paths(df):
    """
    Flatten all image_1/2/3 columns from the given DataFrame
    into a unique list of image paths.
    """
    image_cols = ['image_1_path', 'image_2_path', 'image_3_path']

    all_paths = (
        df[image_cols]
        .stack()          # combine all image columns vertically
        .dropna()         # remove missing entries
        .unique()         # only keep unique paths
        .tolist()         # convert to list
    )

    print(f"📸 Found {len(all_paths)} unique image paths.")
    return all_paths


In [ ]:
def show_case_images(case_id):
    """Display all available images for a given case_id."""
    row = Globals.cases_and_labels_df[Globals.cases_and_labels_df["case_id"] == str(case_id)]
    if row.empty:
        print(f"No case found for ID {case_id}")
        return

    paths = row[['image_1_path', 'image_2_path', 'image_3_path']].dropna(axis=1).values.flatten()
    plt.figure(figsize=(15, 5))
    for i, path in enumerate(paths, 1):
        img = read_image_from_gcs(path)
        plt.subplot(1, len(paths), i)
        plt.imshow(img)
        plt.title(f"{os.path.basename(path)}", fontsize=9)
        plt.axis("off")
    plt.suptitle(f"Case ID: {case_id}", fontsize=12)
    plt.tight_layout()
    plt.show()


In [ ]:
def convert_to_binary_var(col_name):
  df_filtered[col_name].replace('YES', 1, inplace = True)
  df_filtered[col_name].fillna(0, inplace = True)
  print(df_filtered[col_name].value_counts())
  print("")


 ## Image Cleaning

In [ ]:
df_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"
df_filtered = pd.read_csv(df_filtered_path)
df_filtered[:10]

/tmp/ipython-input-932086250.py:2: DtypeWarning: Columns (68,69) have mixed types. Specify dtype option on import or set low_memory=False.
  df_filtered = pd.read_csv(df_filtered_path)


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,condition_weight
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.41
1,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.41
2,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0,0.18
3,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.41
4,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.18
5,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0,0.41
6,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,0.55
7,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,0.23
8,-1003358831658393077,SCIN,1.0.0,2023,AGE_18_TO_29,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,FST4,NaN,NaN,True,True,3.0,4.0,0.23
9,-1003844406100696311,SCIN,1.0.0,2023,AGE_40_TO_49,FEMALE,FST3,0.0,0.0,0.0,...,NaN,NaN,FST1,NaN,NaN,True,True,1.0,1.0,0.33


In [ ]:
from google.cloud import storage
# List all images in dataset/images/
all_images = []
for blob in Globals.gcs_bucket.list_blobs(prefix=Globals.gcs_images_dir):
    if blob.name.endswith((".jpg",".jpeg",".png")):
        all_images.append(blob.name)

In [ ]:
# Problematic images identified by CleanVision
to_drop = [
    "dataset/images/-4593817128438983108.png",
    "dataset/images/-2431769699504014881.png"
]

# Columns that contain image paths
image_cols = ['image_1_path', 'image_2_path', 'image_3_path']

# Build a boolean mask for rows that have any of these images
mask = df_filtered[image_cols].isin(to_drop).any(axis=1)

# Apply filter (keep only rows without those images)
df_filtered = df_filtered[~mask].copy()

In [ ]:
save_dir = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset"
os.makedirs(save_dir, exist_ok=True)

In [ ]:
save_path = os.path.join(save_dir, "image_quality_report.csv")
quality_df = pd.read_csv(save_path)
print("Loaded", len(quality_df), "rows")
quality_df.head()

Loaded 10379 rows


,image_path,blur,brightness_mean,brightness_std,underexp,overexp,contrast,shadow
0,dataset/images/-1001492676369731180.png,49.800900,125.822289,62.991703,0.152014,0.049442,0.988142,0.162389
1,dataset/images/-1001733364362669777.png,238.390468,74.623044,29.900723,0.036180,0.000554,1.000000,0.036180
2,dataset/images/-1003800477193786941.png,89.631693,128.779480,47.623139,0.005774,0.023419,0.980237,0.055794
3,dataset/images/-1005922060850163675.png,4.172010,102.138025,62.690025,0.179942,0.000000,0.990610,0.010904
4,dataset/images/-1007969568196430462.png,9.460231,147.597770,40.054561,0.000000,0.002477,0.819608,0.107458


In [ ]:
# Problematic images identified by CleanVision
to_drop_2 = [
    "dataset/images/4207723573736028617.png",
]

mask_2 = df_filtered[image_cols].isin(to_drop_2).any(axis=1)
df_filtered = df_filtered.loc[~mask_2]

print(f"Removed {mask_2.sum()} rows containing problematic images.")
print(f" Filtered dataset now has {len(df_filtered)} rows.")

Removed 0 rows containing problematic images.
 Filtered dataset now has 8209 rows.


In [ ]:
save_path = os.path.join(save_dir, "image_quality_report_with_sharpness.csv")
quality_df = pd.read_csv(save_path)
print("Loaded", len(quality_df), "rows")
quality_df.head()

Loaded 8209 rows


/tmp/ipython-input-2692060160.py:2: DtypeWarning: Columns (69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  quality_df = pd.read_csv(save_path)


,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_1,dermatologist_gradable_for_fitzpatrick_skin_type_2,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us
0,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
1,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
2,-1000600354148496558,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST2,NaN,NaN,True,True,2.0,1.0
3,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0
4,-1002039107727665188,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,YES,NaN,NaN,FST1,NaN,NaN,True,True,3.0,3.0


# Training Dataset defined


In [ ]:
'''
df_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"
df_filtered = pd.read_csv(df_filtered_path)
df_filtered[:10]
'''

'\ndf_filtered_path = "/content/drive/My Drive/BTT_Skinterest_2A/Dataset/dermatologist_conditions_split.csv"\ndf_filtered = pd.read_csv(df_filtered_path)\ndf_filtered[:10]\n'

In [ ]:
df_filtered.columns.tolist()

['case_id',
 'source',
 'release',
 'year',
 'age_group',
 'sex_at_birth',
 'fitzpatrick_skin_type',
 'race_ethnicity_american_indian_or_alaska_native',
 'race_ethnicity_asian',
 'race_ethnicity_black_or_african_american',
 'race_ethnicity_hispanic_latino_or_spanish_origin',
 'race_ethnicity_middle_eastern_or_north_african',
 'race_ethnicity_native_hawaiian_or_pacific_islander',
 'race_ethnicity_white',
 'race_ethnicity_other_race',
 'race_ethnicity_prefer_not_to_answer',
 'textures_raised_or_bumpy',
 'textures_flat',
 'textures_rough_or_flaky',
 'textures_fluid_filled',
 'body_parts_head_or_neck',
 'body_parts_arm',
 'body_parts_palm',
 'body_parts_back_of_hand',
 'body_parts_torso_front',
 'body_parts_torso_back',
 'body_parts_genitalia_or_groin',
 'body_parts_buttocks',
 'body_parts_leg',
 'body_parts_foot_top_or_side',
 'body_parts_foot_sole',
 'body_parts_other',
 'condition_symptoms_bothersome_appearance',
 'condition_symptoms_bleeding',
 'condition_symptoms_increasing_size',
 'c

In [ ]:
print(df_filtered['fitzpatrick_skin_type'].unique())
print(df_filtered['dermatologist_gradable_for_fitzpatrick_skin_type_1'].unique())
print(df_filtered['dermatologist_gradable_for_fitzpatrick_skin_type_2'].unique()) #fill nan
print(df_filtered['dermatologist_gradable_for_fitzpatrick_skin_type_3'].unique()) #fill nan
print(df_filtered['dermatologist_fitzpatrick_skin_type_label_1'].unique()) #fill nan
print(df_filtered['dermatologist_fitzpatrick_skin_type_label_2'].unique()) #fill nan
print(df_filtered['dermatologist_fitzpatrick_skin_type_label_3'].unique()) #fill nan
print(df_filtered['race_ethnicity_asian'].unique())
print(df_filtered['race_ethnicity_black_or_african_american'].unique())
print(df_filtered['race_ethnicity_hispanic_latino_or_spanish_origin'].unique())
print(df_filtered['race_ethnicity_middle_eastern_or_north_african'].unique())
print(df_filtered['race_ethnicity_american_indian_or_alaska_native'].unique())
print(df_filtered['race_ethnicity_native_hawaiian_or_pacific_islander'].unique())
print(df_filtered['race_ethnicity_white'].unique())
print(df_filtered['race_ethnicity_other_race'].unique())
print(df_filtered['race_ethnicity_prefer_not_to_answer'].unique())
print(df_filtered['other_symptoms_fever'].unique())
print(df_filtered['other_symptoms_chills'].unique())
print(df_filtered['other_symptoms_fatigue'].unique())
print(df_filtered['other_symptoms_joint_pain'].unique())
print(df_filtered['other_symptoms_mouth_sores'].unique())
print(df_filtered['other_symptoms_shortness_of_breath'].unique())
print(df_filtered['other_symptoms_no_relevant_symptoms'].unique())

['NONE_IDENTIFIED' 'FST3' 'FST1' 'FST4' 'FST2' 'FST5' 'FST6']
['YES' 'NO']
[nan 'YES' 'NO']
[nan 'YES' 'NO']
['FST2' 'FST1' 'FST4' 'FST3' 'FST5' nan 'FST6']
[nan 'FST3' 'FST4' 'FST2' 'FST5' 'FST1' 'FST6']
[nan 'FST4' 'FST1' 'FST3' 'FST2' 'FST5']
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[0. 1.]
[1. 0.]


In [ ]:
df_filtered[['dermatologist_gradable_for_fitzpatrick_skin_type_2',
 'dermatologist_gradable_for_fitzpatrick_skin_type_3',
 'dermatologist_fitzpatrick_skin_type_label_1',
 'dermatologist_fitzpatrick_skin_type_label_2',
 'dermatologist_fitzpatrick_skin_type_label_3',
]].isnull().sum()

# going to not use these columns bc too much null data

,0
dermatologist_gradable_for_fitzpatrick_skin_type_2,5245
dermatologist_gradable_for_fitzpatrick_skin_type_3,5245
dermatologist_fitzpatrick_skin_type_label_1,428
dermatologist_fitzpatrick_skin_type_label_2,5469
dermatologist_fitzpatrick_skin_type_label_3,5459


## Baseline Model

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


In [ ]:
# Select highest-confidence condition per case
train_df = df_filtered.copy()

train_df = train_df.loc[train_df.groupby("case_id")["condition_confidence"].idxmax()]
train_df = train_df.reset_index(drop=True)

# Assign top label
train_df["top_label"] = train_df["condition_name"]
train_df

,case_id,source,release,year,age_group,sex_at_birth,fitzpatrick_skin_type,race_ethnicity_american_indian_or_alaska_native,race_ethnicity_asian,race_ethnicity_black_or_african_american,...,dermatologist_gradable_for_fitzpatrick_skin_type_3,dermatologist_fitzpatrick_skin_type_label_1,dermatologist_fitzpatrick_skin_type_label_2,dermatologist_fitzpatrick_skin_type_label_3,gradable_for_monk_skin_tone_india,gradable_for_monk_skin_tone_us,monk_skin_tone_label_india,monk_skin_tone_label_us,condition_weight,top_label
0,-9223126124516060635,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST2,0.0,0.0,0.0,...,NaN,FST1,NaN,NaN,True,True,2.0,2.0,0.33,Scabies
1,-9221035133244224964,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,FST1,NaN,NaN,True,True,2.0,2.0,0.41,Eczema
2,-9218285092875997194,SCIN,1.0.0,2023,AGE_30_TO_39,MALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,FST2,NaN,NaN,True,True,2.0,2.0,1.00,Irritant Contact Dermatitis
3,-9215921701035518354,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,FST2,NaN,NaN,True,True,3.0,3.0,0.33,Tinea
4,-9212359218854294582,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,FST4,NaN,NaN,True,True,2.0,3.0,0.50,Eczema
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3054,9198500737462176368,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,FST3,NaN,NaN,True,True,2.0,2.0,0.50,Eczema
3055,9204674379234810196,SCIN,1.0.0,2023,AGE_UNKNOWN,OTHER_OR_UNSPECIFIED,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,True,True,3.0,4.0,0.67,Urticaria
3056,9210062012872504495,SCIN,1.0.0,2023,AGE_18_TO_29,FEMALE,FST5,0.0,1.0,0.0,...,NaN,FST4,NaN,NaN,True,True,4.0,5.0,0.50,Hypersensitivity
3057,9212720578959463880,SCIN,1.0.0,2023,AGE_40_TO_49,FEMALE,NONE_IDENTIFIED,0.0,0.0,0.0,...,NaN,FST2,NaN,NaN,True,True,2.0,2.0,0.55,Hemangioma


In [ ]:
image_cols = ["image_1_path", "image_2_path", "image_3_path"]

df_long = train_df.melt(
    id_vars=["case_id", "top_label"],
    value_vars=image_cols,
    var_name="image_num",
    value_name="image_path"
)

# Drop rows with missing image paths
df_long = df_long.dropna(subset=["image_path"])
df_long

,case_id,top_label,image_num,image_path
0,-9223126124516060635,Scabies,image_1_path,dataset/images/3422278879386892670.png
1,-9221035133244224964,Eczema,image_1_path,dataset/images/-3162371366187734223.png
2,-9218285092875997194,Irritant Contact Dermatitis,image_1_path,dataset/images/-4063990915557730380.png
3,-9215921701035518354,Tinea,image_1_path,dataset/images/-1590080144874886066.png
4,-9212359218854294582,Eczema,image_1_path,dataset/images/-8500170905733656840.png
...,...,...,...,...
9165,9114324332147869982,Allergic Contact Dermatitis,image_3_path,dataset/images/410063804886627808.png
9166,9114737540227929305,Eczema,image_3_path,dataset/images/-989501707247190818.png
9167,9130094661402000696,Acanthosis nigricans,image_3_path,dataset/images/8581314132129243276.png
9168,9138173567729098672,Acute and chronic dermatitis,image_3_path,dataset/images/2673916593000381599.png


In [ ]:
df_long["image_path"] = "gs://" + Globals.gcs_bucket_name + "/" + df_long["image_path"].astype(str)

In [ ]:
df_long

,case_id,top_label,image_num,image_path
0,-9223126124516060635,Scabies,image_1_path,gs://dx-scin-public-data/dataset/images/342227...
1,-9221035133244224964,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-31623...
2,-9218285092875997194,Irritant Contact Dermatitis,image_1_path,gs://dx-scin-public-data/dataset/images/-40639...
3,-9215921701035518354,Tinea,image_1_path,gs://dx-scin-public-data/dataset/images/-15900...
4,-9212359218854294582,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-85001...
...,...,...,...,...
9165,9114324332147869982,Allergic Contact Dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/410063...
9166,9114737540227929305,Eczema,image_3_path,gs://dx-scin-public-data/dataset/images/-98950...
9167,9130094661402000696,Acanthosis nigricans,image_3_path,gs://dx-scin-public-data/dataset/images/858131...
9168,9138173567729098672,Acute and chronic dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/267391...


In [ ]:
le = LabelEncoder()
df_long["label_id"] = le.fit_transform(df_long["top_label"])
num_classes = len(le.classes_)

print(f"Found {num_classes} unique labels.")


Found 210 unique labels.


In [ ]:
# Remove classes that have fewer than 2 images
label_counts = df_long["label_id"].value_counts()
df_long = df_long[df_long["label_id"].isin(label_counts[label_counts >= 2].index)]

In [ ]:
le = LabelEncoder()
df_long["label_id"] = le.fit_transform(df_long["top_label"])
num_classes = len(le.classes_)

print(f"Found {num_classes} unique labels.")

Found 184 unique labels.


/tmp/ipython-input-35374451.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_long["label_id"] = le.fit_transform(df_long["top_label"])


In [ ]:
df_long

,case_id,top_label,image_num,image_path,label_id
0,-9223126124516060635,Scabies,image_1_path,gs://dx-scin-public-data/dataset/images/342227...,157
1,-9221035133244224964,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-31623...,53
2,-9218285092875997194,Irritant Contact Dermatitis,image_1_path,gs://dx-scin-public-data/dataset/images/-40639...,92
3,-9215921701035518354,Tinea,image_1_path,gs://dx-scin-public-data/dataset/images/-15900...,169
4,-9212359218854294582,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-85001...,53
...,...,...,...,...,...
9165,9114324332147869982,Allergic Contact Dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/410063...,19
9166,9114737540227929305,Eczema,image_3_path,gs://dx-scin-public-data/dataset/images/-98950...,53
9167,9130094661402000696,Acanthosis nigricans,image_3_path,gs://dx-scin-public-data/dataset/images/858131...,4
9168,9138173567729098672,Acute and chronic dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/267391...,11


In [ ]:
# # First compute existence for ALL rows
# df_long["file_exists"] = df_long["image_path"].apply(tf.io.gfile.exists)

# # Extract invalid paths
# invalid_paths = df_long[df_long["file_exists"] == False]["image_path"].tolist()

# print("Number of invalid images:", len(invalid_paths))
# invalid_paths[:20]   # preview first 20

In [ ]:
# dropping invalid file path
idx_drop = df_long[df_long['image_path'] == 'gs://dx-scin-public-data/dataset/images/-2243186711511406658.png'].index

df_long = df_long.drop(idx_drop)
df_long.shape

(6486, 5)

In [ ]:
paths = df_long["image_path"].values
labels = df_long["label_id"].values


train_paths, test_paths, train_labels, test_labels = train_test_split(
    paths, labels,
    test_size=0.4,
    random_state=42,
    stratify=labels
)


In [ ]:
IMG_SIZE = (224, 224)

def load_image(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, IMG_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    label = tf.cast(label, tf.int32)   # IMPORTANT
    return image, label

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_ds = test_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Input(shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)

Epoch 1/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 746s 5s/step - accuracy: 0.0746 - loss: 8.3920 - val_accuracy: 0.1599 - val_loss: 3.9006
Epoch 2/10
122/122 ━━━━━━━━━━━━━━━━━━━━ 666s 5s/step - accuracy: 0.1737 - loss: 3.7723 - val_accuracy: 0.1279 - val_loss: 3.9036
Epoch 3/10
121/122 ━━━━━━━━━━━━━━━━━━━━ 2s 3s/step - accuracy: 0.2689 - loss: 3.2592 

In [ ]:
#Save model
model_save_dir = '/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Sarah_Models'
os.makedirs(model_save_dir, exist_ok=True)
model_path = os.path.join(model_save_dir, 'cnn_dermatology_model.keras')
model.save(model_path)

In [ ]:
#Load model
loaded_model = tf.keras.models.load_model('/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Sarah_Models/cnn_dermatology_model.keras')

In [ ]:
test_loss, test_accuracy = model.evaluate(test_ds, verbose=1)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

In [ ]:
train_loss, train_accuracy = model.evaluate(train_ds, verbose=1)
print(f'Train Loss: {train_loss:.4f}')
print(f'Train Accuracy: {train_accuracy:.4f}')

In [ ]:
# Save label encoder
import pickle
le_save_dir = '/content/drive/My Drive/BTT_Skinterest_2A/Coding for Skinterest Tech 2A Project/Sarah_Models'
os.makedirs(le_save_dir, exist_ok=True)
le_path = os.path.join(le_save_dir, 'label_encoder_baseline.pkl')
with open(le_path, "wb") as f:
    pickle.dump(le, f)

In [ ]:
# Load pickle encoder
import pickle

with open(le_path, "rb") as f:
    le = pickle.load(f)

## Model with Fitzpatrick

In [ ]:
fitzpatrick_cols = [
    'fitzpatrick_skin_type',
    'dermatologist_gradable_for_fitzpatrick_skin_type_1',
]

# extract case_id + body part columns
df_body = train_df[['case_id'] + fitzpatrick_cols].drop_duplicates()

# merge body-part metadata onto df_long rows
df_long_fitzpatrick = df_long
df_long_fitzpatrick = df_long.merge(df_body, on='case_id', how='left')
df_long_fitzpatrick

,case_id,top_label,image_num,image_path,label_id,body_parts_head_or_neck,body_parts_arm,body_parts_palm,body_parts_back_of_hand,body_parts_torso_front,body_parts_torso_back,body_parts_genitalia_or_groin,body_parts_buttocks,body_parts_leg,body_parts_foot_top_or_side,body_parts_foot_sole,body_parts_other
0,-9223126124516060635,Scabies,image_1_path,gs://dx-scin-public-data/dataset/images/342227...,176,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-9221035133244224964,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-31623...,56,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-9218285092875997194,Irritant Contact Dermatitis,image_1_path,gs://dx-scin-public-data/dataset/images/-40639...,99,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-9215921701035518354,Tinea,image_1_path,gs://dx-scin-public-data/dataset/images/-15900...,190,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-9212359218854294582,Eczema,image_1_path,gs://dx-scin-public-data/dataset/images/-85001...,56,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6481,9114324332147869982,Allergic Contact Dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/410063...,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6482,9114737540227929305,Eczema,image_3_path,gs://dx-scin-public-data/dataset/images/-98950...,56,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
6483,9130094661402000696,Acanthosis nigricans,image_3_path,gs://dx-scin-public-data/dataset/images/858131...,4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6484,9138173567729098672,Acute and chronic dermatitis,image_3_path,gs://dx-scin-public-data/dataset/images/267391...,11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
paths = df_long_fitzpatrick["image_path"].values
labels = df_long_fitzpatrick["label_id"].values
metadata = df_long_fitzpatrick[fitzpatrick_cols].astype("float32").values

In [ ]:
# df_long_fitzpatrick["file_exists"] = df_long_fitzpatrick["image_path"].apply(tf.io.gfile.exists)
# invalid_paths = df_long_fitzpatrick[df_long_fitzpatrick["file_exists"] == False]["image_path"].tolist()

# print("Number of invalid images:", len(invalid_paths))
# invalid_paths[:20]


Number of invalid images: 0


[]

In [ ]:
# df_long_fitzpatrick = df_long_fitzpatrick[df_long_fitzpatrick["file_exists"] == True].drop(columns=["file_exists"])

In [ ]:
# Remove classes that have fewer than 2 images
label_counts = df_long_fitzpatrick["label_id"].value_counts()
df_long_fitzpatrick = df_long_fitzpatrick[df_long_fitzpatrick["label_id"].isin(label_counts[label_counts >= 2].index)]
df_long_fitzpatrick

In [ ]:
train_p, test_p, train_l, test_l, train_m, test_m = train_test_split(
    paths, labels, metadata,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

IMG_SIZE = (224, 224)

def load_image_with_meta(path, label, meta):
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_png(img_bytes, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return (img, meta), label


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_p, train_l, train_m))
train_ds = train_ds.map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_p, test_l, test_m))
test_ds = test_ds.map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
image_input = layers.Input(shape=(224, 224, 3))
x = layers.Conv2D(32, 3, activation='relu')(image_input)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)

meta_input = layers.Input(shape=(len(body_part_cols),))
m = layers.Dense(32, activation='relu')(meta_input)

combined = layers.concatenate([x, m])
output = layers.Dense(num_classes, activation='softmax')(combined)

model_fitzpatrick = models.Model(inputs=[image_input, meta_input], outputs=output)
model_fitzpatrick.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model_fitzpatrick.fit(
    train_ds,
    validation_data=test_ds,
    epochs=10
)


Epoch 1/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 61s 285ms/step - accuracy: 0.1363 - loss: 4.6816 - val_accuracy: 0.1641 - val_loss: 3.8381
Epoch 2/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 51s 270ms/step - accuracy: 0.1770 - loss: 3.7069 - val_accuracy: 0.1441 - val_loss: 3.7980
Epoch 3/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 50s 251ms/step - accuracy: 0.2657 - loss: 3.1572 - val_accuracy: 0.1333 - val_loss: 4.0614
Epoch 4/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 53s 274ms/step - accuracy: 0.4974 - loss: 2.1086 - val_accuracy: 0.1094 - val_loss: 5.4015
Epoch 5/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 51s 262ms/step - accuracy: 0.7141 - loss: 1.1593 - val_accuracy: 0.1140 - val_loss: 7.7195
Epoch 6/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 49s 252ms/step - accuracy: 0.8636 - loss: 0.5894 - val_accuracy: 0.1171 - val_loss: 9.6416
Epoch 7/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 48s 250ms/step - accuracy: 0.9301 - loss: 0.3401 - val_accuracy: 0.1071 - val_loss: 10.7941
Epoch 8/10
163/163 ━━━━━━━━━━━━━━━━━━━━ 48s 248ms/step - accuracy: 0.9513 - loss: 

In [ ]:
#Save model
os.makedirs(model_save_dir, exist_ok=True)
model_path_fitzpatrick = os.path.join(model_save_dir, 'cnn_dermatology_model_fitzpatrick.keras')
model.save(model_path_fitzpatrick)

## Model with Race/Ethnicity

In [ ]:
# Chose not to use 'race_ethnicity_prefer_not_to_answer'
race_cols = [
    'race_ethnicity_american_indian_or_alaska_native',
    'race_ethnicity_asian',
    'race_ethnicity_black_or_african_american',
    'race_ethnicity_hispanic_latino_or_spanish_origin',
    'race_ethnicity_middle_eastern_or_north_african',
    'race_ethnicity_native_hawaiian_or_pacific_islander',
    'race_ethnicity_white',
    'race_ethnicity_other_race'
]

# extract case_id + body part columns
df_body = train_df[['case_id'] + race_cols].drop_duplicates()

# merge body-part metadata onto df_long rows
df_long_race = df_long
df_long_race = df_long.merge(df_body, on='case_id', how='left')
df_long_race

In [ ]:
paths = df_long_race["image_path"].values
labels = df_long_race["label_id"].values
metadata = df_long_race[race_cols].astype("float32").values

label_counts = df_long_race["label_id"].value_counts()
df_long_race = df_long_race[df_long_race["label_id"].isin(label_counts[label_counts >= 2].index)]

train_p, test_p, train_l, test_l, train_m, test_m = train_test_split(
    paths, labels, metadata,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

IMG_SIZE = (224, 224)

train_ds = tf.data.Dataset.from_tensor_slices((train_p, train_l, train_m))
train_ds = train_ds.map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_p, test_l, test_m))
test_ds = test_ds.map(load_image_with_meta, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(32).prefetch(tf.data.AUTOTUNE)

